In [1]:
import pandas as pd 
import math
from matplotlib import pyplot as plt

import dw_utils2 

ans_submit = dw_utils2.create_submitter(host='52.91.20.10', port=80, 
                       user="camilo.velez@yuxiglobal.com", # put your full yuxi email address here, including @yuxiglobal.com
                       ws_key="dw3", #this is the workshop key, don't change it 
                       token="BCrUAgei6q8Z" ) #put the token that Mateo sent to you on an e-mail on Wednesday...

Loading dw_utils2 module (v. 20180625)
Creating submitter function Submitting test question to verify connection.
Q00 = 'test answer'
Answer for question Q00 is correct 😃



## Introduction

In this data workshop, we will see some advanced data operatorions using Pandas such as merging data, accessing data through anindex, grouping data, as well as rearranging data to be visualized in a different way. 

Let's load some data frames first:

In [2]:
houses_df = pd.read_csv("house_prices_and_characteristics.csv")

pd.set_option("display.max_columns", 50)

In [ ]:
houses_df.head(5)

## Group By

As its name  implies, this function groups rows of a data frame according to the values in one or more columns.  However, simply calling the function doesn't actually return a `DataFrame` object but rather a `DataFrameGroupBy` object. The latter can be thought of as a dictionary with its keys being the distinct combinations of grouping column values, and each associated value being the group of rows for the column values combination, assembled as a data frame. On a `DataFrameGroupBy` object, one can then apply aggregation operations, such as sum, mean, mode, etc. Keep in mind, that they will only be applied to columns which support such operation, with the other columns being excluded (i.e. non numerical columns being excluded when trying to get the average for each column of the resulting data frame).



In [10]:
grp_by = houses_df.groupby(["MSZoning", "Street"])
type( grp_by  )

pandas.core.groupby.groupby.DataFrameGroupBy

In [ ]:
houses_df.groupby(["Street", "LandContour"]).mean()

Take a look at the `.groups` attribute of the `grp_by` object. Don't be scared by `Int64Index` objects! They are just memory efficent ways to collect the indices of rows in each group...

In [12]:
grp_by.groups

{('C (all)', 'Grvl'): Int64Index([812, 1061], dtype='int64'),
 ('C (all)',
  'Pave'): Int64Index([30, 88, 93, 495, 557, 711, 916, 1279], dtype='int64'),
 ('FV',
  'Pave'): Int64Index([  47,   56,   87,  105,  115,  158,  180,  212,  240,  256,  270,
              281,  285,  297,  317,  377,  381,  399,  409,  412,  453,  460,
              501,  507,  525,  549,  578,  603,  623,  641,  644,  650,  686,
              687,  699,  755,  758,  762,  824,  829,  831,  864,  875,  885,
              914,  959,  973,  975,  977,  989, 1087, 1089, 1091, 1172, 1191,
             1217, 1246, 1265, 1317, 1358, 1364, 1365, 1374, 1442, 1454],
            dtype='int64'),
 ('RH',
  'Pave'): Int64Index([ 341,  383,  543,  635,  671,  681,  778,  840,  913,  951,  955,
             1030, 1206, 1234, 1264, 1326],
            dtype='int64'),
 ('RL', 'Grvl'): Int64Index([335, 582, 1184], dtype='int64'),
 ('RL',
  'Pave'): Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    9,   10,
          

## Groupby Exercise 0

Take `grp_by` and get the number of rows in the group with key `('RH', 'Pave')`

In [35]:
num_rows = None # Replace None with your code
ans_submit( 'Grp0', num_rows )

Grp0 = None
Correct Answer for question Grp0 is not yet defined. Kindly ask Mateo to do it...



The fact that group-by  returns multiple sub-data-frames allows functions to be applied to each individual group, rather than on the entire data frame, which is useful when trying to analyze data in different groups without having the results of one group being affected by data from other groups. 

In the following example two new columns are added to grp_by via an apply function, one of them being the rank (place that a value occupies within a sorted list) and the other being the average price of the top half most expensive houses of each group.

In [ ]:
def enrich_grp( grp ) : 
    """Enrich a group (sub-data-frame within a DataFrameGroupBy) by adding a column that ranks the SalePrices in 
    it and then compute the avg price of the most expensive half of houses. Notice that this number is going to 
    be the same accross all rows! The length (number of rows) of the resulting data frame is the same as the input one"""
    
    grp['RankPrice'] = grp['SalePrice'].rank()
    grp['avg_price_upper_half'] = grp["SalePrice"][ grp["RankPrice"] < len(grp) / 2 ].mean()  
    
    return grp
    
( grp_by.apply( enrich_grp )
        [["SalePrice", "RankPrice", "avg_price_upper_half"]]
        .sort_values( ['avg_price_upper_half']) ).head(20)

## Multiple aggregation functions
The aggregation method `.agg` (_synonym_: `.aggregate`)  in the class `DataFrameGroupBy` allows multiple aggregation operations to be performed on all columns of a data frame at the same time. 

This functions admits two ways of calling it: 

In the first one *a list of aggregation function names* (strings) is passed to it.  This results in all those aggreation functions being applied to all columns of the dataframe (where the aggregation functions applies):

In [ ]:
grp_by.agg(["sum", "mean"])

The operations are only applied on columns which allow them, while the rest of columns are simply ignored from the output. In the following example it can be seen how `aggregated` ends up having less columns that the data frame it came from, `houses_df`, due to the latter having columns which don't allow numerical operations to be performed.

In [ ]:
aggregated = houses_df.groupby(["MSZoning", "Street"]).mean()
print("houses_df has %d columns\naggregated has %d columns" % (houses_df.shape[1],aggregated.shape[1]))

A second way of calling it is passing a dictionary in which the keys are data frame columns and the values are the names of aggregation functions to apply to each column.

In [ ]:
grp_by.agg({
         "LotArea"     : "mean",
         "SalePrice"   : "sum",
         "MSSubClass"  : pd.np.median,    
         # Series.nunique() computes the number of unique values in a series. 
         "MiscVal"     : lambda ser : ser.nunique(),
         # Series.value_counts() returns dict of counts of different values
         "Exterior1st" : lambda ser : " | ".join( ser.value_counts().keys() )             
        }) 

**Now that we've seen some aggregation examples**, a little trick that may be useful when trying to quickly tell how many groups there are in a group by is to do some operation on the groupby object, regardless of what the actual output may be.

## Groupby Exercise 1

How many different streets are there in `houses_df`? (suggestion, do some numerical operation on the groupby)

In [28]:
num_streets = None # Replace None with your code
ans_submit( 'Grp1', num_streets )

Grp1 = None
Answer for question Grp1 is incorrect ☹



## Groupby Exercise 2

How many different combinations of `Street` and `LandContour` are there in `houses_df`?

In [ ]:
num_combinations = None # Replace None with your code
ans_submit( 'Grp2', num_combinations )

**And now for the actual aggregation exercises, in which the output data does matter**

### Aggregation Exercise 0

Group `houses_df` by `Street` and `LotShape` and sum only the lot area of each group

In [ ]:
df = None # Replace None with your code
ans_submit( 'Agg0', df.iloc[2] )

### Aggregation Exercise 1

Redo the same grouping of the previous exercise but instead of getting the sum of lot areas, create a function that gets the _second_ largest lot area in the group, in case the group has only 1 row, return float("NaN")

In [ ]:
df = None # replace None with your code 
ans_submit( 'Agg1',df["LotArea"].sum() )  

In [ ]:
houses_df['RankPrice']  = houses_df['SalePrice'].rank()

(houses_df[['MSZoning', 'Street', 'SalePrice', 'RankPrice']]
       .sort_values( ['SalePrice' ] ).head(15) )

## Indexing

An **index** in a data frame is a set of values that play essentially the same role as the keys in a dictionary. An index is usually composed of integers, strings or tuples. To each value of the index, there is an associated row or, possibly, a *set of rows* in the data frame. Further, accessing that row or set of rows is an efficient (O(1)) operation that *does not* require to scan the whole data set. It's important to note that the index is *not considered* a column in the dataset as such. It's in a different category. However, an index can be easily. 

### set_index

Every data frame has an index. When the dataframe is first created from scratch, this index is just a sequential index containing the integers from 0 to `len(df)-1`. 
However it is often more useful to define and index from the values of a column, this is done by means of 

This function makes turns the specified column the new index of the data frame by either taking it out of the data columns (default) or keeping it both as the index as well as a data column (by setting the drop parameter to False). More than one column may be set as index by inputing a list of columns.

In the following examples we shall work on (a subset of) the Gowalla checkins dataset. 
This dataset contains check-in events by users of the Gowalla social networking site to set of locations. 

In [5]:
checkins_df = ( pd.read_csv("check_in_gowalla.csv")                  
                  .rename( columns={"check_in_time" : "checkin_time", 'location_id' : "location" })
                  .sort_values( 'location' ))
friends_df = pd.read_csv("friends.csv")  #PENDING: mover esto para más abajo cuando se necesite
users_a = pd.read_csv("users_a.csv")
users_b = pd.read_csv("users_b.csv")

The following is what the `checkins_df` looks like

In [ ]:
checkins_df.head(15)

Notice that there is a column of apparently random numbers  on the let that is not named. That's actually not a column, but the index. Notice that, when we defined the dataframe, we sorted it by `location` right after loading it from the csv. The first record in the resulting order by `location` actually comes from the 826-th row of the `csv`. That's where the 826 in front of the first record comes from.  

Looking at the `.index` attribute we see that `checkins_df` has an index consisting of integers, in the order just shown.

In [4]:
checkins_df.index 

NameError: name 'checkins_df' is not defined

In [ ]:
chins_indexed_1 = checkins_df.set_index( 'location' )
chins_indexed_1.head( 20 )

In [ ]:
chins_indexed_1.index

Again, the index consists of integers but they come from `location` column in the original data frame.

It's important to see that setting an index on a data frame doesn't actually change the original data frame at all, rather it creates a new one with the specified index. 

In [ ]:
checkins_df.head(10)

Not let's define and index on two columns!

In [177]:
chins_indexed_2 = checkins_df.set_index( ['location', 'user'] )
chins_indexed_2.head( 20 )

checkin_time  longitude   latitude
location user                                        
9101     346    12/22/2009 9:18  38.956833 -95.244792
         3842    9/3/2010 10:37  38.956833 -95.244792
         65     9/11/2010 16:24  38.956833 -95.244792
         346    4/29/2010 17:35  38.956833 -95.244792
         346     3/4/2010 18:53  38.956833 -95.244792
         346    2/17/2010 19:33  38.956833 -95.244792
         346   12/22/2009 18:29  38.956833 -95.244792
         346   12/22/2009 14:34  38.956833 -95.244792
         346    12/21/2009 9:18  38.956833 -95.244792
         346   12/16/2009 19:02  38.956833 -95.244792
         346   12/17/2009 19:11  38.956833 -95.244792
         346   12/20/2009 23:32  38.956833 -95.244792
         346   12/18/2009 18:35  38.956833 -95.244792
9401     4717   6/24/2009 12:50  32.939640 -97.076869
         491   12/20/2009 19:14  32.939640 -97.076869
9501     4806  11/17/2009 16:33  30.272291 -97.745790
         1174   9/30/2010 22:39  30.272291 -97.745790
         3667    9/7/2010 17:56  30.272291 -97.745790
         3273  11/14/2009 16:41  30.272291 -97.745790
         335   10/16/2009 11:19  30.272291 -97.745790

The resulting index is called a _hierarchical_ index because it has a hierarchy of levels. 

In this case , the first level groups  records by location an the second by user.

** Exercise I0: ** 

The following line generates an error. Copy an paste the *last* (non-empty) line of the error message into the answer. 
Make sure to understand **why** this error is produced. If you don't understand, discuss it with your instructor. 

In [178]:
chins_indexed_1['location']

KeyError: 'location'

In [ ]:
ans_submit( "I0", "copy and paste the last (non-empty) line of the error message")

It is also possible to keep a column as data as well as setting it as and index by  passing the `drop = False` to `set_index`.

In [ ]:
chins_indexed_1 = checkins_df.set_index("location", drop=False)
chins_indexed_1.head( 20 )

### Retrieving rows through an index -- basic usage


The primary purpose of an index is to *efficiently* retrieve a row or set of rows from a data frame. This is done throught the `DataFrame.loc` accessor ( 'loc' is short for 'locate' ). We will go in depth into the usage of `loc`, but for now the essential usage is as follows:

In [ ]:
first_row = checkins_df.loc[826]
first_row

** Exercise I1: ** 

What is the _type_ of `first_row`?  (You can use the built-in function `type()` to answer this question)

In [ ]:
ans_submit( "I1", "pandas.....fill-in rest of the type (fully-qualified) name")

In [ ]:
first_row_v2 = checkins_df.loc[ [826] ]
first_row_v2

This is the same as before except that it is nicely formatted (why?)


**Exercise I2**

What is the type of `first_row_v2`? 

In [ ]:
ans_submit( "I2", "pandas.....")

In [ ]:
another_row = checkins_df.loc[ [0] ]
another_row

Notice that a *single* index value can map to _many_ rows...

In [ ]:
chins_indexed_1.loc[1038401]

** Exercise I3 **

How many rows are there for location = 9501

In [ ]:
ans_submit( "I3", ... ) #your answer instead of ...

### Accessing rows when there is a hierarchichal index 

When there are _n_ levels in the index, you can specify 1, 2 or up to 'n' values in to the `.loc` accessor 

In [ ]:
chins_indexed_2.loc[ 9101 ]

In [ ]:
chins_indexed_2.loc[  (9101, 346 ), ]

In order to avoid that nasty warning we are careful to sort both levels of the index..

In [ ]:
chins_indexed_2s = chins_indexed_2.sort_index( level= [0,1], axis=0)  
chins_indexed_2s.loc[ (9101,346) ]

### reset_index

It reset the index to an auto-incremental one and makes any previously set index a data column again. By default it resets all previously indexed columns, but the parameter level allows only certain indexes to be reset

In [ ]:
chins_indexed_2.reset_index().head(20)

In [ ]:
chins_indexed_2.reset_index(level=1).head(20)

## Exercise R0

Reset the index of `chins_indexed_2s` by one level and count how many users have been on location 9101

In [ ]:
ans = None # replace none with your answer
ans_submit( 'R0', ans  )

## iloc and loc

Pandas' main way of referencing data is through iloc and loc functions. Both functions are similar but only superficially so.

`iloc` is simpler, it's basic usage syntax is as follows

```
   df.iloc[ list_of_ints_r, list_of_ints_c ]  
```

Here `list_of_ints_r` specifies the 0-based indices of _rows_ in the dataframe's own order and and `list_of_ints_c` specifies the 0-based indices of `columns`. Despite its name `iloc` does not take into account the dataframe's index at all! The `i` in `iloc` stands for `integer`. 

Instead of `list_of_ints_r`, you can also put a slice, such as `10:47` to get rows numbered 10 through 46 (not including 47 as is usual with Python slicing). Remember that just writing `:47` is a shorthand for `0:47`  and `10:` is a shorthand for `10:len(df)`. Similarly just writing `:` means `0:len(df)`. 

Analogous considerations hold for the second argument, `list_of_ints_c`. 


Now onto `loc`.

The `.loc` method (accessor)  receives as input list of row and column *labels*. The row labels are the keys defined in the index. The colum `labels` are the regular column names you already know and love. 


The basic syntax is:

```
   df.iloc[ list_of_labels_r, list_of_labels_c ]  
```



In [ ]:
#PENDING: En los siguientes ejemplos usar un data frame con un índice definido y que no sea numérico. 
# Si no, puede haber mucha confusión... El caso de dataframe con índice numérico se puede discutir más abajo como un caso especial...

In [ ]:
houses_df2 = houses_df.set_index("Street")
houses_df2.iloc[[1,2,3],[0, 1,3]]

In [ ]:
houses_df2.iloc[ 1:7, 0:5 ]

In [ ]:
houses_df2.loc[["Pave"],["MSSubClass","LotFrontage"]].head(10)

In [ ]:
houses_df2 = houses_df2.reset_index()
houses_df2.head(10)

## Pivot

The pivot function generatates a new dataframe that presents  rearranging a column from a given data  frame in the form of a grid, which may be clearer to analyze. 
The function receives three basic arguments> 

  * **index**: specifies the column whose values will become the index of the new data frame.
  * **columns** : a column whose values are going to be turned into column headers in the new another column as the columns of the new data frame. It is advisable for this particular column not to have many different values so that the new data frame won't have a lot of columns.
  * ** values ** : A columns whose are going to be displayed in the grid. Although a list of columns can be used for values values, doing so would be the same as simply pivoting 2 different values and concatenating the result.
  
All this will be a lot clearer with an example. 

First we generate an aggregated version of our houses data frame to get average prices and areas for all the various combinations of Exterior1st and Street.

In [ ]:
houses_agg = ( houses_df.groupby(["Exterior1st", "Street"])
                        .agg( {"SalePrice" : "mean", "LotArea" : "mean"})
                        .reset_index()
                        .sort_values('Street')) 
houses_agg

Notice that only some of the values of 'Exterior1st' appeared for `Street = 'Grvl'`, hence the grid contains NaNs for those.  

In [ ]:
houses_agg.pivot(index='Exterior1st', columns="Street", values='SalePrice').head(10)

## Pivot Exercise 0

Take the houses_df data frame, group it by MSZoning and LotShape, get the average SalePrice and pivot it with LotShape and MSZoning as index and columns respectively (to use either LotShape or MSZoning to pivot, you have to reset the index of the group by)


In [ ]:
df = None # replace None with your code 
ans_submit( 'Piv0',df.iloc[1,2]  )

## Merge and join

Both functions allow data from different data frames to be combined into according to a 'crossing' or 'look-up' rule. 

Although both `merge` and `join` do really a similar thing, the way they do it is different. The function `merge`is pandas default function for joining data, it's basically *pandas counterpart of SQL's join*, and requires the specification of which columns of both data frames would be compared. Merge doesn't care at all about about the indexes defined on them. 

On the other hand, Panda's `join function is more of a convenience thing (it even uses merge internally), joining is basically doing a merge by taking advantage of the indexes of both data frames. 

The following figure summarizes the different 4 types or merge: _ inner, outer, left and right_.

In [ ]:
from IPython.core.display import HTML
HTML( '<img src="merge.png" height="200" width="800"/>') 

## Inner merge

Is the default merge in case the `how` parameter is not specified. It yields rows for which there are matching values of the specified merge columns on both data_Frames.

In [ ]:
users_a.merge( users_b, left_on='subject_id', right_on='subject_id')

## Left merge

It gkeeps all the data from the first data frame, adding data from the second one whenever there is a row matching and filling with `NaN` the missing columns from the second data frame in which no match was found

In [ ]:
users_a.merge( users_b, how="left", left_on='subject_id', right_on='subject_id')
# a.merge(b, left_on='subject_id', right_on='subject_id', how="left") #alternative syntax

## Right merge

It's pretty much the same exact thing as left merge with the data frame on the left taking being on the right and viceversa. It keeps all the data from the second data frame adding data from the first one whenever they intersect and filling with `NaN` the missing columns from the first data frame if no match was found

In [ ]:
users_a.merge( users_b,  how="right", left_on='subject_id', right_on='subject_id')
# a.merge(b, left_on='subject_id', right_on='subject_id', how="right") #alternative syntax

## Outer merge

It's basically the combination of both left and right join, keeping all the data from both data frames and filling out with NaN if no match found was found for either.

In [ ]:
users_a.merge( users_b, how="outer", left_on='subject_id', right_on='subject_id')
# a.merge(b, left_on='subject_id', right_on='subject_id', how="outer") #alternative syntax

## PENDING: more exercises with merge ... el que está no está tan chévere....

### Join

Sometimes, when data frames are already indexed by the same thing, it is a lot easier to use `join`, which uses matches rows from two them according to their index value. 

To see this, let's define two dataframes indexed by 'location' 

In [ ]:
counts_by_loc = ( checkins_df.groupby( "location" )
                             .agg( { "user" : "nunique",
                                     "check_in_time" : "count"}) )
counts_by_loc.head(10)

In [ ]:
lat_lon_by_loc = ( checkins_df[['location', 'latitude', 'longitude']]
                              .drop_duplicates()
                              .set_index('location') ) 
lat_lon_by_loc.head(10)

As the both dataframes are indexed on the same id, it is very easy to just join them by that id.

In [8]:
counts_by_loc.join( lat_lon_by_loc ).head(10)

,user,checkin_time,longitude,latitude,location
826,346,12/22/2009 9:18,38.956833,-95.244792,9101
8151,3842,9/3/2010 10:37,38.956833,-95.244792,9101
99,65,9/11/2010 16:24,38.956833,-95.244792,9101
808,346,4/29/2010 17:35,38.956833,-95.244792,9101
815,346,3/4/2010 18:53,38.956833,-95.244792,9101
818,346,2/17/2010 19:33,38.956833,-95.244792,9101
824,346,12/22/2009 18:29,38.956833,-95.244792,9101
825,346,12/22/2009 14:34,38.956833,-95.244792,9101
827,346,12/21/2009 9:18,38.956833,-95.244792,9101
832,346,12/16/2009 19:02,38.956833,-95.244792,9101


A non so conventional way of merging data is by merging a data frame with itself as to compare some rows with one another, here's an example of it.

### A non-conventional merge: self-merge

In [6]:
chins2 = checkins_df.merge( checkins_df, on="location" )
chins3 = chins2[ (chins2.checkin_time_x < chins2.checkin_time_y) & 
                 (chins2.user_x != chins2.user_y )]
chins4 = ( chins3[["user_x", "user_y", "location"]]
                 .drop_duplicates()                 
                 .groupby(["user_x", "user_y"])
                 .agg( {"location" : "count"})
                 .rename( columns = { "location" : "location_count" } )
                 .reset_index() )

chins4.sort_values('location_count', ascending=False).head(40)

,user_x,user_y,location_count
5341,1404,1080,5
13869,4806,4717,5
10216,3460,2616,4
9888,3412,2211,4
5342,1404,1148,4
12737,4542,1146,4
12523,4496,4686,4
10641,3631,4542,3
2173,392,447,3
3707,680,337,3


Here we merged the checkins data frame with itself every time an user went to a location some other user had gone as well. We then filter the rows in which an user arrived after another one, not at the same time (given that the previous merge rows matched with themselves). Finally the information was grouped for a better visualization.

## Merge Exercise 0

Perform an inner merge between the first 100 rows and the following 100 rows of `houses_df` comparing the `Street` column

In [ ]:
df = None # replace None with your code 
ans_submit( 'Mer0',df.iloc[:,1].sum())  

## Shift

The shift function is pandas' way of lagging a series, which is useful when trying to see how something changes as time goes on, like predicting a future value for certain series, or studying how a given series affects another one

In [182]:
chins_u4771 = checkins_df[ checkins_df.user == 4771 ].copy()
chins_u4771.head(15)

,user,checkin_time,longitude,latitude,location
9397,4771,6/28/2010 20:10,40.777194,-74.355921,116401
9378,4771,8/24/2010 18:11,40.777194,-74.355921,116401
9398,4771,6/21/2010 17:49,40.777194,-74.355921,116401
9399,4771,6/20/2010 16:57,40.777194,-74.355921,116401
9400,4771,6/19/2010 17:56,40.777194,-74.355921,116401
9377,4771,9/3/2010 18:26,40.777194,-74.355921,116401
9401,4771,6/15/2010 16:50,40.777194,-74.355921,116401
9403,4771,6/13/2010 17:10,40.777194,-74.355921,116401
9404,4771,6/7/2010 15:24,40.777194,-74.355921,116401
9396,4771,7/3/2010 17:12,40.777194,-74.355921,116401


In [186]:
chins_u4771['checkin_tm_prev'] = chins_u4771['checkin_time'].shift(1)
chins_u4771['checkin_tm_next'] = chins_u4771['checkin_time'].shift(-1)
chins_u4771.head(15)

,user,checkin_time,longitude,latitude,location,checkin_tm_prev,checkin_tm_next
9397,4771,6/28/2010 20:10,40.777194,-74.355921,116401,NaN,8/24/2010 18:11
9378,4771,8/24/2010 18:11,40.777194,-74.355921,116401,6/28/2010 20:10,6/21/2010 17:49
9398,4771,6/21/2010 17:49,40.777194,-74.355921,116401,8/24/2010 18:11,6/20/2010 16:57
9399,4771,6/20/2010 16:57,40.777194,-74.355921,116401,6/21/2010 17:49,6/19/2010 17:56
9400,4771,6/19/2010 17:56,40.777194,-74.355921,116401,6/20/2010 16:57,9/3/2010 18:26
9377,4771,9/3/2010 18:26,40.777194,-74.355921,116401,6/19/2010 17:56,6/15/2010 16:50
9401,4771,6/15/2010 16:50,40.777194,-74.355921,116401,9/3/2010 18:26,6/13/2010 17:10
9403,4771,6/13/2010 17:10,40.777194,-74.355921,116401,6/15/2010 16:50,6/7/2010 15:24
9404,4771,6/7/2010 15:24,40.777194,-74.355921,116401,6/13/2010 17:10,7/3/2010 17:12
9396,4771,7/3/2010 17:12,40.777194,-74.355921,116401,6/7/2010 15:24,5/30/2010 16:29


In the previous example the shift function was really poorly used, given that by simple shifting every register by one position the first register for every user (apart from user 0) is literally the last register from the previous user which is obviously false information. To avoid doing so, it would be best to first group up the data by user, that being said...

## Shift Exercise 0

Shift only the check in time of every user in the `checkins_df` by 1 position (as a sugestion do a groupby and apply the same technique as with enrich_grp function above...) 

In [187]:
df = None # replace None with your code 
ans_submit( 'Shift0',math.isnan(df.iloc[2,-1])  )

AttributeError: 'NoneType' object has no attribute 'iloc'

## Aggregation Exercise 3

Use `checkins_df` to get the number of visits and datetime of the first visit for each location

In [67]:
checkins_df

,user,checkin_time,longitude,latitude,location
826,346,12/22/2009 9:18,38.956833,-95.244792,9101
8151,3842,9/3/2010 10:37,38.956833,-95.244792,9101
99,65,9/11/2010 16:24,38.956833,-95.244792,9101
808,346,4/29/2010 17:35,38.956833,-95.244792,9101
815,346,3/4/2010 18:53,38.956833,-95.244792,9101
818,346,2/17/2010 19:33,38.956833,-95.244792,9101
824,346,12/22/2009 18:29,38.956833,-95.244792,9101
825,346,12/22/2009 14:34,38.956833,-95.244792,9101
827,346,12/21/2009 9:18,38.956833,-95.244792,9101
832,346,12/16/2009 19:02,38.956833,-95.244792,9101


In [72]:
checkins_df.groupby('location').agg({"checkin_time":[min, max]}).columns

MultiIndex(levels=[['checkin_time'], ['min', 'max']],
           labels=[[0, 0], [0, 1]])

In [65]:
agg3_df = None # replace None with your code 
ans_submit( 'Agg31',agg3_df['user'].sum())
ans_submit( 'Agg32',agg3_df['checkin_time'].loc[9101])

TypeError: 'NoneType' object is not subscriptable

## Aggregation Exercise 4

Use `checkins_df` to get the amount of diferent visitors per location (suggestion: search for the nunique method)

In [64]:
agg4_df = None # replace None with your code 
ans_submit( 'Agg4',agg4_df['user'].loc[9101])

TypeError: 'NoneType' object is not subscriptable